# Project Setup

### Importing the required libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Rescaling, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.metrics import Precision, Recall, Accuracy
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score,log_loss
from tabulate import tabulate
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

### Declaring the Variables

In [3]:
IMAGE_SIZE = [299,299]
BATCH_SIZE=16
CLASSES=["L1S3","L2S3","L3S3","LC1","LC4","LC5","LC6","LC9"]
NUMBER_OF_CLASSES=len(CLASSES)

In [4]:
training_image_dir='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/001_Dataset/Training'
testing_image_dir='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/001_Dataset/Testing'
validation_image_dir='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/001_Dataset/Validation'

### Fetching and Preprocessing the image files

In [5]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [6]:
training_image_data_set =tf.keras.utils.image_dataset_from_directory(
  training_image_dir,
  label_mode='int',
  seed=131,
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE
)

Found 8000 files belonging to 8 classes.


In [7]:
validation_image_data_set=tf.keras.utils.image_dataset_from_directory(
    validation_image_dir,
    label_mode='int',
    seed=131,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

Found 2400 files belonging to 8 classes.


In [8]:
testing_image_data_set=tf.keras.utils.image_dataset_from_directory(
    testing_image_dir,
    label_mode='int',
    seed=131,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

Found 2400 files belonging to 8 classes.


In [9]:
print(f'Using {len(training_image_data_set)} sets for Training')
print(f'Using {len(validation_image_data_set)} sets for Validation')
print(f'Using {len(testing_image_data_set)} sets for Testing')

Using 500 sets for Training
Using 150 sets for Validation
Using 150 sets for Testing


In [10]:
normalized_training_set=training_image_data_set.map(lambda x,y: (normalization_layer(x), y))
normalized_validation_set=validation_image_data_set.map(lambda x,y: (normalization_layer(x), y))
normalized_testing_set=testing_image_data_set.map(lambda x,y: (normalization_layer(x), y))

# image_batch, labels_batch = next(iter(normalized_training_set))      #<-----the data is fethched in this form: image batch, label batch

# Training the Scratch Model

### Building the CNN Model

In [13]:
def Build_Model():
  model=Sequential()

  model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(299,299,3)))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Conv2D(32, (3,3), 1, activation='relu'))
  model.add(MaxPooling2D())

  model.add(Conv2D(64, (3,3), 1, activation='relu'))
  model.add(MaxPooling2D())

  model.add(Conv2D(128, (3,3), 1, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Conv2D(256, (3,3), 1, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Flatten())

  model.add(Dense(256, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(NUMBER_OF_CLASSES, activation='softmax'))

  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  return model

In [14]:
# Summary of the model
model=Build_Model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 297, 297, 16)      448       
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 148, 148, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 146, 146, 32)      4640      
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 35, 35, 64)       

### Peforming the 10 Fold Cross Validation

In [15]:

# Logs
logdir='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/003_Logs/10FoldCrossValidatedModel_Logs'

# Model Directory
model_dir='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/004_Cross_Validated_Models/'

In [16]:
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [17]:
FOLDS=10
SCORES_PER_FOLD=[]

In [18]:
TFold_start=time.time()

In [19]:
i=0
j=50
k=450
for fold_var in range(0,FOLDS,1):
  print(f'Fold {fold_var+1}')
  print('--------------------------------------------------------------------------------------------')
  training_set1=normalized_training_set.take(i)
  training_set2=normalized_training_set.skip(i+j).take(k)
  training_set=training_set1.concatenate(training_set2)
  validation_set=normalized_training_set.skip(i).take(j)
  i=i+50
  j=50
  k=k-50

  model=Build_Model()
  Model_Run = model.fit(
    training_set,
    epochs=20,
    steps_per_epoch=len(training_set),
    callbacks=[tensorboard_callback]
  )
  print('\n')
  print(f'Evaluating the model')
  print('\n')
  cur_scores=model.evaluate(validation_set)
  cur_scores[0]*=100
  cur_scores[1]*=100
  cur_scores=[fold_var+1,cur_scores[0],cur_scores[1]]
  SCORES_PER_FOLD.append(cur_scores)

  model.save(model_dir+'model_'+str(fold_var+1)+'.h5')
  tf.keras.backend.clear_session()
  print('--------------------------------------------------------------------------------------------')

Fold 1
--------------------------------------------------------------------------------------------
Epoch 1/20
450/450 [==============================] - 1340s 3s/step - loss: 1.3515 - accuracy: 0.3611
Epoch 2/20
450/450 [==============================] - 31s 63ms/step - loss: 0.9653 - accuracy: 0.5260
Epoch 3/20
450/450 [==============================] - 31s 63ms/step - loss: 0.8485 - accuracy: 0.5758
Epoch 4/20
450/450 [==============================] - 32s 64ms/step - loss: 0.7869 - accuracy: 0.6037
Epoch 5/20
450/450 [==============================] - 33s 65ms/step - loss: 0.8499 - accuracy: 0.5842
Epoch 6/20
450/450 [==============================] - 32s 64ms/step - loss: 0.7617 - accuracy: 0.6122
Epoch 7/20
450/450 [==============================] - 31s 62ms/step - loss: 0.6668 - accuracy: 0.6685
Epoch 8/20
450/450 [==============================] - 31s 62ms/step - loss: 0.6084 - accuracy: 0.7011
Epoch 9/20
450/450 [==============================] - 31s 63ms/step - loss: 0.5983 -

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


--------------------------------------------------------------------------------------------
Fold 2
--------------------------------------------------------------------------------------------
Epoch 1/20
450/450 [==============================] - 39s 81ms/step - loss: 1.3183 - accuracy: 0.3639
Epoch 2/20
450/450 [==============================] - 34s 75ms/step - loss: 0.9068 - accuracy: 0.4997
Epoch 3/20
450/450 [==============================] - 34s 74ms/step - loss: 0.7887 - accuracy: 0.5953
Epoch 4/20
450/450 [==============================] - 34s 74ms/step - loss: 0.6880 - accuracy: 0.6350
Epoch 5/20
450/450 [==============================] - 37s 81ms/step - loss: 0.6301 - accuracy: 0.6696
Epoch 6/20
450/450 [==============================] - 33s 73ms/step - loss: 0.5762 - accuracy: 0.6876
Epoch 7/20
450/450 [==============================] - 35s 76ms/step - loss: 0.5510 - accuracy: 0.7092
Epoch 8/20
450/450 [==============================] - 34s 74ms/step - loss: 0.5826 - accuracy

In [20]:
TFold_stop=time.time()

In [21]:
print(f'Time required for 10 Fold Cross Validation= {TFold_stop-TFold_start} seconds')

Time required for 10 Fold Cross Validation= 12475.340168952942 seconds


In [22]:
column_headers=['Fold Number','Validation Loss(%)','Validation Accuracy(%)']
print('10 Fold Cross Validation Result')
print(tabulate(SCORES_PER_FOLD, headers=column_headers, tablefmt="grid"))

10 Fold Cross Validation Result
+---------------+----------------------+--------------------------+
|   Fold Number |   Validation Loss(%) |   Validation Accuracy(%) |
+===============+======================+==========================+
|             1 |              63.8118 |                   75.75  |
+---------------+----------------------+--------------------------+
|             2 |              56.8098 |                   76.625 |
+---------------+----------------------+--------------------------+
|             3 |              68.8331 |                   76.375 |
+---------------+----------------------+--------------------------+
|             4 |              42.2307 |                   79.5   |
+---------------+----------------------+--------------------------+
|             5 |              54.5564 |                   79.125 |
+---------------+----------------------+--------------------------+
|             6 |              55.8589 |                   80     |
+---------------

### Selecting the model with highest accuracy for training

In [23]:
max_acc=SCORES_PER_FOLD[0][2]
max_acc_index=[]
for i in range(len(SCORES_PER_FOLD)):
  if(SCORES_PER_FOLD[i][2]>max_acc):
    max_acc=SCORES_PER_FOLD[i][2]
print(f'The maximum validation accuracy is: {max_acc}')
for i in range(len(SCORES_PER_FOLD)):
  if(SCORES_PER_FOLD[i][2]==max_acc):
    max_acc_index.append(i)
print(f'The folds having {max_acc} validation accuracy are:')
for i in range(len(max_acc_index)):
  print(max_acc_index[i]+1)

result=-1
if(len(max_acc_index)>1):
    min_loss=SCORES_PER_FOLD[max_acc_index[0]][1]
    j=0
    result=max_acc_index[0]
    while(j<len(max_acc_index)):
      index=max_acc_index[j]
      if(SCORES_PER_FOLD[index][1]<min_loss):
        min_loss=SCORES_PER_FOLD[index][1]
        result=index
      j+=1
else:
  result=max_acc_index[0]
print(f'Selecting the model of fold no {result+1} for fine parameter tuning')
model=load_model(model_dir+'model_'+str(result+1)+'.h5')

The maximum validation accuracy is: 81.62500262260437
The folds having 81.62500262260437 validation accuracy are:
8
Selecting the model of fold no 8 for fine parameter tuning


### Training the model for fine parameter tuning

In [ ]:
# Log Directory
scratch_model_logdir='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/003_Logs/ScratchModel_Logs'

# Fine tuned Model Directory
scratch_model_dir='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/005_Fine_Tuned_Model/FineTuned_Model.h5'

# Generated Data Diagrams Directory
scratch_model_loss='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/ScratchModel/Loss'
scratch_model_accuracy='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/ScratchModel/Accuracy'
scratch_model_rocauc='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/ScratchModel/Roc_Auc'
scratch_model_rocauc_zoomed='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/ScratchModel/Roc_Auc_Zoomed'
scratch_model_confusion_matrix='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/ScratchModel/Confusion_Matrix'

In [ ]:
scratch_model_tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=scratch_model_logdir)

In [ ]:
Training_start=time.time()

In [ ]:
Cur_Model_Run = model.fit(
    normalized_training_set,
    validation_data=normalized_validation_set,
    epochs=30,
    validation_steps=len(normalized_validation_set),
    callbacks=[scratch_model_tensorboard_callback]
  )

In [ ]:
Training_stop=time.time()

In [ ]:
print(f'Time required for model fine parameter tuning= {Training_stop-Training_start} seconds')

### Saving the fine tuned model

In [ ]:
model.save(scratch_model_dir)

### Plotting the performance of the model after fine parameter tuning

In [ ]:
# Plotting the loss


fig=plt.figure(figsize=(10,6))
plt.plot(Cur_Model_Run.history['loss'], color='blue', label='Training Loss')
plt.plot(Cur_Model_Run.history['val_loss'], color='red', label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
fig.suptitle('Loss',fontsize=15)
plt.legend()
plt.show()
fig.savefig(scratch_model_loss + '.eps', format='eps', dpi=600)
fig.savefig(scratch_model_loss + '.png', format='png', dpi=600)

In [ ]:
# Plotting the accuracy


fig=plt.figure(figsize=(10,6))
plt.plot(Cur_Model_Run.history['accuracy'], color='blue', label='Training Accuracy')
plt.plot(Cur_Model_Run.history['val_accuracy'], color='red', label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
fig.suptitle('Accuracy',fontsize=15)
plt.legend()
plt.show()
fig.savefig(scratch_model_accuracy + '.eps', format='eps', dpi=600)
fig.savefig(scratch_model_accuracy + '.png', format='png', dpi=600)

### Calcualting Precision, Recall, Accuracy, F Score, Cohen-Kappa Score,ROC AUC and Log-Loss Score of the Model

In [ ]:
pre = Precision()
re = Recall()
acc = Accuracy()

In [ ]:
True_label=[]
Predicted_label=[]
Predicted_probabilities=[]
score=[]
for images,labels in (normalized_testing_set):
  score.append(model.evaluate(images,labels))
  for it in range(len(labels)):
    True_label.append(labels[it])
  pred=model.predict(images)
  y_max=[]
  for it in range(len(pred)):
      Predicted_probabilities.append(pred[it])
      y_max.append(np.argmax(pred[it]))
      Predicted_label.append(np.argmax(pred[it]))
  pre.update_state(labels,y_max)
  re.update_state(labels,y_max)
  acc.update_state(labels,y_max)

In [ ]:
f_score=(2*pre.result().numpy()*re.result().numpy())/(pre.result().numpy()+re.result().numpy())

In [ ]:
kappa=cohen_kappa_score(True_label,Predicted_label)

In [ ]:
True_label=np.array(True_label)
Predicted_label=np.array(Predicted_label)
Predicted_probabilities=np.array(Predicted_probabilities)

In [ ]:
Binarized_True_label=label_binarize(True_label,classes=[0, 1, 2, 3, 4])

In [ ]:
#Calcualting the fpr, tpr and auc


fpr = [0] * NUMBER_OF_CLASSES
tpr = [0] * NUMBER_OF_CLASSES
roc_auc= [0] * NUMBER_OF_CLASSES
avg_roc_auc=0
for i in range(NUMBER_OF_CLASSES):
    fpr[i], tpr[i], _ = roc_curve(Binarized_True_label[:, i], Predicted_probabilities[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
roc_auc_details=[]
for i in range(NUMBER_OF_CLASSES):
  temp=[CLASSES[i],roc_auc[i]]
  roc_auc_details.append(temp)

In [ ]:
column_headers=['CLASS','ROC-AUC']
print('ROC-AUC Results')
print(tabulate(roc_auc_details, headers=column_headers, tablefmt="grid"))

In [ ]:
avg_roc_auc=sum(roc_auc)/NUMBER_OF_CLASSES

In [ ]:
print(f'The average ROC-AUC value is: {avg_roc_auc}')

In [ ]:
#Plotting the ROC Curve


fig=plt.figure(figsize=(10,10))
color=['red','green','blue','cyan','magenta','yellow','orange','purple']
for i in range(NUMBER_OF_CLASSES):
  plt.plot(fpr[i],tpr[i],color[i],label='Class {} (AUC= {:.6F})'.format(CLASSES[i],roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('One-vs-Rest ROC Curve',fontsize=15)
plt.legend(loc='lower right')
plt.show()
fig.savefig(scratch_model_rocauc + '.eps', format='eps', dpi=600)
fig.savefig(scratch_model_rocauc + '.png', format='png', dpi=600)

In [ ]:
#Plotting the ROC Curve (Zoomed in at the top left corner)


fig=plt.figure(figsize=(10,10))
color=['red','green','blue','cyan','magenta','yellow','orange','purple']
for i in range(NUMBER_OF_CLASSES):
  plt.plot(fpr[i],tpr[i],color[i],label='Class {} (AUC= {:.6F})'.format(CLASSES[i],roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 0.50])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('One-vs-Rest ROC Curve (Zoomed top-left)',fontsize=15)
plt.legend(loc='lower right')
plt.show()
fig.savefig(scratch_model_rocauc_zoomed + '.eps', format='eps', dpi=600)
fig.savefig(scratch_model_rocauc_zoomed + '.png', format='png', dpi=600)

In [ ]:
#Calculating the Log-loss Score (One-vs-Rest)


temp=[0]*NUMBER_OF_CLASSES
loss=[0]*NUMBER_OF_CLASSES
epsilon = 1e-15  # Small constant to avoid division by zero
N = Binarized_True_label.shape[0]

for i in range(NUMBER_OF_CLASSES):
  # Clip predicted probabilities to avoid log(0) cases
  temp[i]= np.clip(Predicted_probabilities[:,i], epsilon, 1 - epsilon)
  # Calculate cross-entropy loss
  loss[i] = -np.sum(Binarized_True_label[:,i] * np.log(temp[i])) / N

In [ ]:
log_loss_result=[]
for i in range(NUMBER_OF_CLASSES):
  log_loss_result.append([CLASSES[i],loss[i]])

In [ ]:
column_headers=['CLASS','LOG-LOSS VALUE']
print('Log-Loss Results')
print(tabulate(log_loss_result, headers=column_headers, tablefmt="grid"))

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}, F Score: {f_score}, Cohen Kappa Score: {kappa}')

In [ ]:
#Plotting the Confusion Matrix


confusion_matrix=metrics.confusion_matrix(True_label,Predicted_label,normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["L1S3","L2S3","L3S3","LC1","LC4","LC5","LC6","LC9"])
fig, ax = plt.subplots(figsize=(10,10))
cm_display.plot(ax=ax)
plt.show()
fig.savefig(scratch_model_confusion_matrix + '.eps', format='eps', dpi=600)
fig.savefig(scratch_model_confusion_matrix + '.png', format='png', dpi=600)

### Demonstrating how to use the model

In [ ]:
img=image.load_img('/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/001_Dataset/Testing/LC4/LC4_dl_1009.jpg',target_size=(299,299))

In [ ]:
x=image.img_to_array(img)

In [ ]:
x=x/255

In [ ]:
x=np.expand_dims(x,axis=0)
x.shape

In [ ]:
pred=model.predict(x)

In [ ]:
print(pred)

In [ ]:
pred_name = CLASSES[np.argmax(pred)]
print(pred_name)

In [ ]:
temp_testing_set=normalized_testing_set.take(1)

pred_start=time.time()
for (images,labels) in (temp_testing_set):
  result=model.predict(images)
pred_stop=time.time()

In [ ]:
print(f'Time requrired for giving prediction on one set of {len(labels)} images= {pred_stop-pred_start}')
print(f'Average time requrired for giving prediction on one image= {(pred_stop-pred_start)/len(labels)}')

# Training the Pretrained Models

### Log Directories

In [ ]:
# Vgg16
vgg16_logdir='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/003_Logs/Vgg16_Logs'

# MobileNetV2
mobilenetv2_logdir='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/003_Logs/MobileNetV2_Logs'

# ResNet50
resnet50_logdir='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/003_Logs/ResNet50_Logs'

# EfficientNetB0
efficientnetb0_logdir='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/003_Logs/EfficientNetB0_Logs'


### Generated Data Diagrams Directories

In [ ]:
# Vgg16
vgg16_loss='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/Vgg16/Loss'
vgg16_accuracy='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/Vgg16/Accuracy'
vgg16_rocauc='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/Vgg16/Roc_Auc'
vgg16_rocauc_zoomed='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/Vgg16/Roc_Auc_Zoomed'
vgg16_confusion_matrix='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/Vgg16/Confusion_Matrix'

# MobileNetV2
mobilenetv2_loss='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/MobileNetV2/Loss'
mobilenetv2_accuracy='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/MobileNetV2/Accuracy'
mobilenetv2_rocauc='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/MobileNetV2/Roc_Auc'
mobilenetv2_rocauc_zoomed='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/MobileNetV2/Roc_Auc_Zoomed'
mobilenetv2_confusion_matrix='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/MobileNetV2/Confusion_Matrix'

# ResNet50
resnet50_loss='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/ResNet50/Loss'
resnet50_accuracy='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/ResNet50/Accuracy'
resnet50_rocauc='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/ResNet50/Roc_Auc'
resnet50_rocauc_zoomed='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/ResNet50/Roc_Auc_Zoomed'
resnet50_confusion_matrix='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/ResNet50/Confusion_Matrix'

# EfficientNetB0
efficientnetb0_loss='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/EfficientNetB0/Loss'
efficientnetb0_accuracy='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/EfficientNetB0/Accuracy'
efficientnetb0_rocauc='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/EfficientNetB0/Roc_Auc'
efficientnetb0_rocauc_zoomed='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/EfficientNetB0/Roc_Auc_Zoomed'
efficientnetb0_confusion_matrix='/content/drive/MyDrive/Conference_Journal/2024_Journal_Cardamom_CNN_Work/006_Generated_Data_Diagrams/EfficientNetB0/Confusion_Matrix'

### Training the VGG16 Model and evaluating the performance

In [ ]:
from keras.applications import VGG16

In [ ]:
vgg16_tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=vgg16_logdir)

In [ ]:
vgg16_base=VGG16(weights='imagenet',include_top=False,input_shape=(299,299,3))
vgg16_model=Sequential()
vgg16_model.add(vgg16_base)
vgg16_model.add(Flatten())
vgg16_model.add(Dense(NUMBER_OF_CLASSES,activation='softmax'))

vgg16_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
vgg16_model.summary()

In [ ]:
vgg16_Training_start=time.time()

In [ ]:
vgg16_Model_Run = vgg16_model.fit(
    normalized_training_set,
    validation_data=normalized_validation_set,
    epochs=30,
    validation_steps=len(normalized_validation_set),
    callbacks=[vgg16_tensorboard_callback]
  )

In [ ]:
vgg16_Training_stop=time.time()

In [ ]:
print(f'Time required for VGG16 model training= {vgg16_Training_stop-vgg16_Training_start} seconds')

In [ ]:
# Plotting the loss


fig=plt.figure(figsize=(10,6))
plt.plot(vgg16_Model_Run.history['loss'], color='blue', label='Training Loss')
plt.plot(vgg16_Model_Run.history['val_loss'], color='red', label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
fig.suptitle('Loss',fontsize=15)
plt.legend()
plt.show()
fig.savefig(vgg16_loss + '.eps', format='eps', dpi=600)
fig.savefig(vgg16_loss + '.png', format='png', dpi=600)

In [ ]:
# Plotting the accuracy


fig=plt.figure(figsize=(10,6))
plt.plot(vgg16_Model_Run.history['accuracy'], color='blue', label='Training Accuracy')
plt.plot(vgg16_Model_Run.history['val_accuracy'], color='red', label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
fig.suptitle('Accuracy',fontsize=15)
plt.legend()
plt.show()
fig.savefig(vgg16_accuracy + '.eps', format='eps', dpi=600)
fig.savefig(vgg16_accuracy + '.png', format='png', dpi=600)

In [ ]:
# Calculating the Precision, Recall, Accuracy

pre = Precision()
re = Recall()
acc = Accuracy()

In [ ]:
True_label=[]
Predicted_label=[]
Predicted_probabilities=[]
score=[]
for images,labels in (normalized_testing_set):
  score.append(vgg16_model.evaluate(images,labels))
  for it in range(len(labels)):
    True_label.append(labels[it])
  pred=vgg16_model.predict(images)
  y_max=[]
  for it in range(len(pred)):
      Predicted_probabilities.append(pred[it])
      y_max.append(np.argmax(pred[it]))
      Predicted_label.append(np.argmax(pred[it]))
  pre.update_state(labels,y_max)
  re.update_state(labels,y_max)
  acc.update_state(labels,y_max)

In [ ]:
f_score=(2*pre.result().numpy()*re.result().numpy())/(pre.result().numpy()+re.result().numpy())

In [ ]:
kappa=cohen_kappa_score(True_label,Predicted_label)

In [ ]:
True_label=np.array(True_label)
Predicted_label=np.array(Predicted_label)
Predicted_probabilities=np.array(Predicted_probabilities)

In [ ]:
Binarized_True_label=label_binarize(True_label,classes=[0, 1, 2, 3, 4])

In [ ]:
#Calcualting the fpr, tpr and auc


fpr = [0] * NUMBER_OF_CLASSES
tpr = [0] * NUMBER_OF_CLASSES
roc_auc= [0] * NUMBER_OF_CLASSES
avg_roc_auc=0
for i in range(NUMBER_OF_CLASSES):
    fpr[i], tpr[i], _ = roc_curve(Binarized_True_label[:, i], Predicted_probabilities[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
roc_auc_details=[]
for i in range(NUMBER_OF_CLASSES):
  temp=[CLASSES[i],roc_auc[i]]
  roc_auc_details.append(temp)

In [ ]:
column_headers=['CLASS','ROC-AUC']
print('ROC-AUC Results')
print(tabulate(roc_auc_details, headers=column_headers, tablefmt="grid"))

In [ ]:
avg_roc_auc=sum(roc_auc)/NUMBER_OF_CLASSES

In [ ]:
print(f'The average ROC-AUC value is: {avg_roc_auc}')

In [ ]:
#Plotting the ROC Curve


fig=plt.figure(figsize=(10,10))
color=['red','green','blue','cyan','magenta','yellow','orange','purple']
for i in range(NUMBER_OF_CLASSES):
  plt.plot(fpr[i],tpr[i],color[i],label='Class {} (AUC= {:.6F})'.format(CLASSES[i],roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('One-vs-Rest ROC Curve',fontsize=15)
plt.legend(loc='lower right')
plt.show()
fig.savefig(vgg16_rocauc + '.eps', format='eps', dpi=600)
fig.savefig(vgg16_rocauc + '.png', format='png', dpi=600)

In [ ]:
#Plotting the ROC Curve (Zoomed in at the top left corner)


fig=plt.figure(figsize=(10,10))
color=['red','green','blue','cyan','magenta','yellow','orange','purple']
for i in range(NUMBER_OF_CLASSES):
  plt.plot(fpr[i],tpr[i],color[i],label='Class {} (AUC= {:.6F})'.format(CLASSES[i],roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 0.50])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('One-vs-Rest ROC Curve (Zoomed top-left)',fontsize=15)
plt.legend(loc='lower right')
plt.show()
fig.savefig(vgg16_rocauc_zoomed + '.eps', format='eps', dpi=600)
fig.savefig(vgg16_rocauc_zoomed + '.png', format='png', dpi=600)

In [ ]:
#Calculating the Log-loss Score (One-vs-Rest)


temp=[0]*NUMBER_OF_CLASSES
loss=[0]*NUMBER_OF_CLASSES
epsilon = 1e-15  # Small constant to avoid division by zero
N = Binarized_True_label.shape[0]

for i in range(NUMBER_OF_CLASSES):
  # Clip predicted probabilities to avoid log(0) cases
  temp[i]= np.clip(Predicted_probabilities[:,i], epsilon, 1 - epsilon)
  # Calculate cross-entropy loss
  loss[i] = -np.sum(Binarized_True_label[:,i] * np.log(temp[i])) / N

In [ ]:
log_loss_result=[]
for i in range(NUMBER_OF_CLASSES):
  log_loss_result.append([CLASSES[i],loss[i]])

In [ ]:
column_headers=['CLASS','LOG-LOSS VALUE']
print('Log-Loss Results')
print(tabulate(log_loss_result, headers=column_headers, tablefmt="grid"))

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}, F Score: {f_score}, Cohen Kappa Score: {kappa}')

In [ ]:
#Plotting the Confusion Matrix


confusion_matrix=metrics.confusion_matrix(True_label,Predicted_label,normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["L1S3","L2S3","L3S3","LC1","LC4","LC5","LC6","LC9"])
fig, ax = plt.subplots(figsize=(10,10))
cm_display.plot(ax=ax)
plt.show()
fig.savefig(vgg16_confusion_matrix + '.eps', format='eps', dpi=600)
fig.savefig(vgg16_confusion_matrix + '.png', format='png', dpi=600)

### Training the  MobileNetV2 Model and evaluating the performance

In [ ]:
from keras.applications import MobileNetV2

In [ ]:
mobilenetv2_tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=mobilenetv2_logdir)

In [ ]:
mobilenetv2_base=MobileNetV2(weights='imagenet',include_top=False,input_shape=(299,299,3))
mobilenetv2_model=Sequential()
mobilenetv2_model.add(mobilenetv2_base)
mobilenetv2_model.add(Flatten())
mobilenetv2_model.add(Dense(NUMBER_OF_CLASSES,activation='softmax'))

mobilenetv2_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
mobilenetv2_model.summary()

In [ ]:
mobilenetv2_Training_start=time.time()

In [ ]:
mobilenetv2_Model_Run=mobilenetv2_model.fit(
    normalized_training_set,
    validation_data=normalized_validation_set,
    epochs=30,
    validation_steps=len(normalized_validation_set),
    callbacks=[mobilenetv2_tensorboard_callback]
)

In [ ]:
mobilenetv2_Training_stop=time.time()

In [ ]:
print(f'Time required for MOBILENETV2 model training= {mobilenetv2_Training_stop-mobilenetv2_Training_start} seconds')

In [ ]:
# Plotting the loss


fig=plt.figure(figsize=(10,6))
plt.plot(mobilenetv2_Model_Run.history['loss'], color='blue', label='Training Loss')
plt.plot(mobilenetv2_Model_Run.history['val_loss'], color='red', label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
fig.suptitle('Loss',fontsize=15)
plt.legend()
plt.show()
fig.savefig(mobilenetv2_loss + '.eps', format='eps', dpi=600)
fig.savefig(mobilenetv2_loss + '.png', format='png', dpi=600)

In [ ]:
# Plotting the accuracy


fig=plt.figure(figsize=(10,6))
plt.plot(mobilenetv2_Model_Run.history['accuracy'], color='blue', label='Training Accuracy')
plt.plot(mobilenetv2_Model_Run.history['val_accuracy'], color='red', label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
fig.suptitle('Accuracy',fontsize=15)
plt.legend()
plt.show()
fig.savefig(mobilenetv2_accuracy + '.eps', format='eps', dpi=600)
fig.savefig(mobilenetv2_accuracy + '.png', format='png', dpi=600)

In [ ]:
# Calculating the Precision, Recall, Accuracy

pre = Precision()
re = Recall()
acc = Accuracy()

In [ ]:
True_label=[]
Predicted_label=[]
Predicted_probabilities=[]
score=[]
for images,labels in (normalized_testing_set):
  score.append(mobilenetv2_model.evaluate(images,labels))
  for it in range(len(labels)):
    True_label.append(labels[it])
  pred=mobilenetv2_model.predict(images)
  y_max=[]
  for it in range(len(pred)):
      Predicted_probabilities.append(pred[it])
      y_max.append(np.argmax(pred[it]))
      Predicted_label.append(np.argmax(pred[it]))
  pre.update_state(labels,y_max)
  re.update_state(labels,y_max)
  acc.update_state(labels,y_max)

In [ ]:
f_score=(2*pre.result().numpy()*re.result().numpy())/(pre.result().numpy()+re.result().numpy())

In [ ]:
kappa=cohen_kappa_score(True_label,Predicted_label)

In [ ]:
True_label=np.array(True_label)
Predicted_label=np.array(Predicted_label)
Predicted_probabilities=np.array(Predicted_probabilities)

In [ ]:
Binarized_True_label=label_binarize(True_label,classes=[0, 1, 2, 3, 4])

In [ ]:
#Calcualting the fpr, tpr and auc


fpr = [0] * NUMBER_OF_CLASSES
tpr = [0] * NUMBER_OF_CLASSES
roc_auc= [0] * NUMBER_OF_CLASSES
avg_roc_auc=0
for i in range(NUMBER_OF_CLASSES):
    fpr[i], tpr[i], _ = roc_curve(Binarized_True_label[:, i], Predicted_probabilities[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
roc_auc_details=[]
for i in range(NUMBER_OF_CLASSES):
  temp=[CLASSES[i],roc_auc[i]]
  roc_auc_details.append(temp)

In [ ]:
column_headers=['CLASS','ROC-AUC']
print('ROC-AUC Results')
print(tabulate(roc_auc_details, headers=column_headers, tablefmt="grid"))

In [ ]:
avg_roc_auc=sum(roc_auc)/NUMBER_OF_CLASSES

In [ ]:
print(f'The average ROC-AUC value is: {avg_roc_auc}')

In [ ]:
#Plotting the ROC Curve


fig=plt.figure(figsize=(10,10))
color=['red','green','blue','cyan','magenta','yellow','orange','purple']
for i in range(NUMBER_OF_CLASSES):
  plt.plot(fpr[i],tpr[i],color[i],label='Class {} (AUC= {:.6F})'.format(CLASSES[i],roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('One-vs-Rest ROC Curve',fontsize=15)
plt.legend(loc='lower right')
plt.show()
fig.savefig(mobilenetv2_rocauc + '.eps', format='eps', dpi=600)
fig.savefig(mobilenetv2_rocauc + '.png', format='png', dpi=600)

In [ ]:
#Plotting the ROC Curve (Zoomed in at the top left corner)


fig=plt.figure(figsize=(10,10))
color=['red','green','blue','cyan','magenta','yellow','orange','purple']
for i in range(NUMBER_OF_CLASSES):
  plt.plot(fpr[i],tpr[i],color[i],label='Class {} (AUC= {:.6F})'.format(CLASSES[i],roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 0.50])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('One-vs-Rest ROC Curve (Zoomed top-left)',fontsize=15)
plt.legend(loc='lower right')
plt.show()
fig.savefig(mobilenetv2_rocauc_zoomed + '.eps', format='eps', dpi=600)
fig.savefig(mobilenetv2_rocauc_zoomed + '.png', format='png', dpi=600)

In [ ]:
#Calculating the Log-loss Score (One-vs-Rest)


temp=[0]*NUMBER_OF_CLASSES
loss=[0]*NUMBER_OF_CLASSES
epsilon = 1e-15  # Small constant to avoid division by zero
N = Binarized_True_label.shape[0]

for i in range(NUMBER_OF_CLASSES):
  # Clip predicted probabilities to avoid log(0) cases
  temp[i]= np.clip(Predicted_probabilities[:,i], epsilon, 1 - epsilon)
  # Calculate cross-entropy loss
  loss[i] = -np.sum(Binarized_True_label[:,i] * np.log(temp[i])) / N

In [ ]:
log_loss_result=[]
for i in range(NUMBER_OF_CLASSES):
  log_loss_result.append([CLASSES[i],loss[i]])

In [ ]:
column_headers=['CLASS','LOG-LOSS VALUE']
print('Log-Loss Results')
print(tabulate(log_loss_result, headers=column_headers, tablefmt="grid"))

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}, F Score: {f_score}, Cohen Kappa Score: {kappa}')

In [ ]:
#Plotting the Confusion Matrix


confusion_matrix=metrics.confusion_matrix(True_label,Predicted_label,normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["L1S3","L2S3","L3S3","LC1","LC4","LC5","LC6","LC9"])
fig, ax = plt.subplots(figsize=(10,10))
cm_display.plot(ax=ax)
plt.show()
fig.savefig(mobilenetv2_confusion_matrix + '.eps', format='eps', dpi=600)
fig.savefig(mobilenetv2_confusion_matrix + '.png', format='png', dpi=600)

### Training the ResNet50 Model and evaluating the performance

In [ ]:
from keras.applications import ResNet50

In [ ]:
resnet50_tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=resnet50_logdir)

In [ ]:
resnet50_base=ResNet50(weights='imagenet',include_top=False,input_shape=(299,299,3))
resnet50_model=Sequential()
resnet50_model.add(resnet50_base)
resnet50_model.add(Flatten())
resnet50_model.add(Dense(NUMBER_OF_CLASSES,activation='softmax'))

resnet50_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
resnet50_model.summary()

In [ ]:
resnet50_Training_start=time.time()

In [ ]:
resnet50_Model_Run=resnet50_model.fit(
    normalized_training_set,
    validation_data=normalized_validation_set,
    epochs=30,
    validation_steps=len(normalized_validation_set),
    callbacks=[resnet50_tensorboard_callback]
)

In [ ]:
resnet50_Training_stop=time.time()

In [ ]:
print(f'Time required for RESNET50 model training= {resnet50_Training_stop-resnet50_Training_start} seconds')

In [ ]:
# Plotting the loss


fig=plt.figure(figsize=(10,6))
plt.plot(resnet50_Model_Run.history['loss'], color='blue', label='Training Loss')
plt.plot(resnet50_Model_Run.history['val_loss'], color='red', label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
fig.suptitle('Loss',fontsize=15)
plt.legend()
plt.show()
fig.savefig(resnet50_loss + '.eps', format='eps', dpi=600)
fig.savefig(resnet50_loss + '.png', format='png', dpi=600)

In [ ]:
# Plotting the accuracy


fig=plt.figure(figsize=(10,6))
plt.plot(resnet50_Model_Run.history['accuracy'], color='blue', label='Training Accuracy')
plt.plot(resnet50_Model_Run.history['val_accuracy'], color='red', label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
fig.suptitle('Accuracy',fontsize=15)
plt.legend()
plt.show()
fig.savefig(resnet50_accuracy + '.eps', format='eps', dpi=600)
fig.savefig(resnet50_accuracy + '.png', format='png', dpi=600)

In [ ]:
# Calculating the Precision, Recall, Accuracy

pre = Precision()
re = Recall()
acc = Accuracy()

In [ ]:
True_label=[]
Predicted_label=[]
Predicted_probabilities=[]
score=[]
for images,labels in (normalized_testing_set):
  score.append(resnet50_model.evaluate(images,labels))
  for it in range(len(labels)):
    True_label.append(labels[it])
  pred=resnet50_model.predict(images)
  y_max=[]
  for it in range(len(pred)):
      Predicted_probabilities.append(pred[it])
      y_max.append(np.argmax(pred[it]))
      Predicted_label.append(np.argmax(pred[it]))
  pre.update_state(labels,y_max)
  re.update_state(labels,y_max)
  acc.update_state(labels,y_max)

In [ ]:
f_score=(2*pre.result().numpy()*re.result().numpy())/(pre.result().numpy()+re.result().numpy())

In [ ]:
kappa=cohen_kappa_score(True_label,Predicted_label)

In [ ]:
True_label=np.array(True_label)
Predicted_label=np.array(Predicted_label)
Predicted_probabilities=np.array(Predicted_probabilities)

In [ ]:
Binarized_True_label=label_binarize(True_label,classes=[0, 1, 2, 3, 4])

In [ ]:
#Calcualting the fpr, tpr and auc


fpr = [0] * NUMBER_OF_CLASSES
tpr = [0] * NUMBER_OF_CLASSES
roc_auc= [0] * NUMBER_OF_CLASSES
avg_roc_auc=0
for i in range(NUMBER_OF_CLASSES):
    fpr[i], tpr[i], _ = roc_curve(Binarized_True_label[:, i], Predicted_probabilities[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
roc_auc_details=[]
for i in range(NUMBER_OF_CLASSES):
  temp=[CLASSES[i],roc_auc[i]]
  roc_auc_details.append(temp)

In [ ]:
column_headers=['CLASS','ROC-AUC']
print('ROC-AUC Results')
print(tabulate(roc_auc_details, headers=column_headers, tablefmt="grid"))

In [ ]:
avg_roc_auc=sum(roc_auc)/NUMBER_OF_CLASSES

In [ ]:
print(f'The average ROC-AUC value is: {avg_roc_auc}')

In [ ]:
#Plotting the ROC Curve


fig=plt.figure(figsize=(10,10))
color=['red','green','blue','cyan','magenta','yellow','orange','purple']
for i in range(NUMBER_OF_CLASSES):
  plt.plot(fpr[i],tpr[i],color[i],label='Class {} (AUC= {:.6F})'.format(CLASSES[i],roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('One-vs-Rest ROC Curve',fontsize=15)
plt.legend(loc='lower right')
plt.show()
fig.savefig(resnet50_rocauc + '.eps', format='eps', dpi=600)
fig.savefig(resnet50_rocauc + '.png', format='png', dpi=600)

In [ ]:
#Plotting the ROC Curve (Zoomed in at the top left corner)


fig=plt.figure(figsize=(10,10))
color=['red','green','blue','cyan','magenta','yellow','orange','purple']
for i in range(NUMBER_OF_CLASSES):
  plt.plot(fpr[i],tpr[i],color[i],label='Class {} (AUC= {:.6F})'.format(CLASSES[i],roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 0.50])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('One-vs-Rest ROC Curve (Zoomed top-left)',fontsize=15)
plt.legend(loc='lower right')
plt.show()
fig.savefig(resnet50_rocauc_zoomed + '.eps', format='eps', dpi=600)
fig.savefig(resnet50_rocauc_zoomed + '.ong', format='png', dpi=600)

In [ ]:
#Calculating the Log-loss Score (One-vs-Rest)


temp=[0]*NUMBER_OF_CLASSES
loss=[0]*NUMBER_OF_CLASSES
epsilon = 1e-15  # Small constant to avoid division by zero
N = Binarized_True_label.shape[0]

for i in range(NUMBER_OF_CLASSES):
  # Clip predicted probabilities to avoid log(0) cases
  temp[i]= np.clip(Predicted_probabilities[:,i], epsilon, 1 - epsilon)
  # Calculate cross-entropy loss
  loss[i] = -np.sum(Binarized_True_label[:,i] * np.log(temp[i])) / N

In [ ]:
log_loss_result=[]
for i in range(NUMBER_OF_CLASSES):
  log_loss_result.append([CLASSES[i],loss[i]])

In [ ]:
column_headers=['CLASS','LOG-LOSS VALUE']
print('Log-Loss Results')
print(tabulate(log_loss_result, headers=column_headers, tablefmt="grid"))

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}, F Score: {f_score}, Cohen Kappa Score: {kappa}')

In [ ]:
#Plotting the Confusion Matrix


confusion_matrix=metrics.confusion_matrix(True_label,Predicted_label,normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["L1S3","L2S3","L3S3","LC1","LC4","LC5","LC6","LC9"])
fig, ax = plt.subplots(figsize=(10,10))
cm_display.plot(ax=ax)
plt.show()
fig.savefig(resnet50_confusion_matrix + '.eps', format='eps', dpi=600)
fig.savefig(resnet50_confusion_matrix + '.png', format='png', dpi=600)

### Training the EfficientNetB0 Model and evaluating the performance

In [ ]:
from keras.applications import EfficientNetB0

In [ ]:
efficientnetb0_tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=efficientnetb0_logdir)

In [ ]:
efficientnetb0_base=EfficientNetB0(weights='imagenet',include_top=False,input_shape=(299,299,3))
efficientnetb0_model=Sequential()
efficientnetb0_model.add(efficientnetb0_base)
efficientnetb0_model.add(Flatten())
efficientnetb0_model.add(Dense(NUMBER_OF_CLASSES,activation='softmax'))

efficientnetb0_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
efficientnetb0_model.summary()

In [ ]:
efficientnetb0_Training_start=time.time()

In [ ]:
efficientnetb0_Model_Run=efficientnetb0_model.fit(
    normalized_training_set,
    validation_data=normalized_validation_set,
    epochs=30,
    validation_steps=len(normalized_validation_set),
    callbacks=[efficientnetb0_tensorboard_callback]
)

In [ ]:
efficientnetb0_Training_stop=time.time()

In [ ]:
print(f'Time requied for EFFICIENTNETB0 model training= {efficientnetb0_Training_stop-efficientnetb0_Training_start} seconds')

In [ ]:
# Plotting the loss


fig=plt.figure(figsize=(10,6))
plt.plot(efficientnetb0_Model_Run.history['loss'], color='blue', label='Training Loss')
plt.plot(efficientnetb0_Model_Run.history['val_loss'], color='red', label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
fig.suptitle('Loss',fontsize=15)
plt.legend()
plt.show()
fig.savefig(efficientnetb0_loss + '.eps', format='eps', dpi=600)
fig.savefig(efficientnetb0_loss + '.png', format='png', dpi=600)

In [ ]:
# Plotting the accuracy


fig=plt.figure(figsize=(10,6))
plt.plot(efficientnetb0_Model_Run.history['accuracy'], color='blue', label='Training Accuracy')
plt.plot(efficientnetb0_Model_Run.history['val_accuracy'], color='red', label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
fig.suptitle('Accuracy',fontsize=15)
plt.legend()
plt.show()
fig.savefig(efficientnetb0_accuracy + '.eps', format='eps', dpi=600)
fig.savefig(efficientnetb0_accuracy + '.png', format='png', dpi=600)

In [ ]:
# Calculating the Precision, Recall, Accuracy

pre = Precision()
re = Recall()
acc = Accuracy()

In [ ]:
True_label=[]
Predicted_label=[]
Predicted_probabilities=[]
score=[]
for images,labels in (normalized_testing_set):
  score.append(efficientnetb0_model.evaluate(images,labels))
  for it in range(len(labels)):
    True_label.append(labels[it])
  pred=efficientnetb0_model.predict(images)
  y_max=[]
  for it in range(len(pred)):
      Predicted_probabilities.append(pred[it])
      y_max.append(np.argmax(pred[it]))
      Predicted_label.append(np.argmax(pred[it]))
  pre.update_state(labels,y_max)
  re.update_state(labels,y_max)
  acc.update_state(labels,y_max)

In [ ]:
f_score=(2*pre.result().numpy()*re.result().numpy())/(pre.result().numpy()+re.result().numpy())

In [ ]:
kappa=cohen_kappa_score(True_label,Predicted_label)

In [ ]:
True_label=np.array(True_label)
Predicted_label=np.array(Predicted_label)
Predicted_probabilities=np.array(Predicted_probabilities)

In [ ]:
Binarized_True_label=label_binarize(True_label,classes=[0, 1, 2, 3, 4])

In [ ]:
#Calcualting the fpr, tpr and auc


fpr = [0] * NUMBER_OF_CLASSES
tpr = [0] * NUMBER_OF_CLASSES
roc_auc= [0] * NUMBER_OF_CLASSES
avg_roc_auc=0
for i in range(NUMBER_OF_CLASSES):
    fpr[i], tpr[i], _ = roc_curve(Binarized_True_label[:, i], Predicted_probabilities[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
roc_auc_details=[]
for i in range(NUMBER_OF_CLASSES):
  temp=[CLASSES[i],roc_auc[i]]
  roc_auc_details.append(temp)

In [ ]:
column_headers=['CLASS','ROC-AUC']
print('ROC-AUC Results')
print(tabulate(roc_auc_details, headers=column_headers, tablefmt="grid"))

In [ ]:
avg_roc_auc=sum(roc_auc)/NUMBER_OF_CLASSES

In [ ]:
print(f'The average ROC-AUC value is: {avg_roc_auc}')

In [ ]:
#Plotting the ROC Curve


fig=plt.figure(figsize=(10,10))
color=['red','green','blue','cyan','magenta','yellow','orange','purple']
for i in range(NUMBER_OF_CLASSES):
  plt.plot(fpr[i],tpr[i],color[i],label='Class {} (AUC= {:.6F})'.format(CLASSES[i],roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('One-vs-Rest ROC Curve',fontsize=15)
plt.legend(loc='lower right')
plt.show()
fig.savefig(efficientnetb0_rocauc + '.eps', format='eps', dpi=600)
fig.savefig(efficientnetb0_rocauc + '.png', format='png', dpi=600)

In [ ]:
#Plotting the ROC Curve (Zoomed in at the top left corner)


fig=plt.figure(figsize=(10,10))
color=['red','green','blue','cyan','magenta','yellow','orange','purple']
for i in range(NUMBER_OF_CLASSES):
  plt.plot(fpr[i],tpr[i],color[i],label='Class {} (AUC= {:.6F})'.format(CLASSES[i],roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 0.50])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('One-vs-Rest ROC Curve (Zoomed top-left)',fontsize=15)
plt.legend(loc='lower right')
plt.show()
fig.savefig(efficientnetb0_rocauc_zoomed + '.eps', format='eps', dpi=600)
fig.savefig(efficientnetb0_rocauc_zoomed + '.png', format='png', dpi=600)

In [ ]:
#Calculating the Log-loss Score (One-vs-Rest)


temp=[0]*NUMBER_OF_CLASSES
loss=[0]*NUMBER_OF_CLASSES
epsilon = 1e-15  # Small constant to avoid division by zero
N = Binarized_True_label.shape[0]

for i in range(NUMBER_OF_CLASSES):
  # Clip predicted probabilities to avoid log(0) cases
  temp[i]= np.clip(Predicted_probabilities[:,i], epsilon, 1 - epsilon)
  # Calculate cross-entropy loss
  loss[i] = -np.sum(Binarized_True_label[:,i] * np.log(temp[i])) / N

In [ ]:
log_loss_result=[]
for i in range(NUMBER_OF_CLASSES):
  log_loss_result.append([CLASSES[i],loss[i]])

In [ ]:
column_headers=['CLASS','LOG-LOSS VALUE']
print('Log-Loss Results')
print(tabulate(log_loss_result, headers=column_headers, tablefmt="grid"))

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}, F Score: {f_score}, Cohen Kappa Score: {kappa}')

In [ ]:
#Plotting the Confusion Matrix


confusion_matrix=metrics.confusion_matrix(True_label,Predicted_label,normalize='true')
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["L1S3","L2S3","L3S3","LC1","LC4","LC5","LC6","LC9"])
fig, ax = plt.subplots(figsize=(10,10))
cm_display.plot(ax=ax)
plt.show()
fig.savefig(efficientnetb0_confusion_matrix + '.eps', format='eps', dpi=600)
fig.savefig(efficientnetb0_confusion_matrix + '.png', format='png', dpi=600)